In [1]:
import os
import sys
import torch
from transformers import BertTokenizer
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

In [2]:
sys.path.append(os.path.join(os.getcwd(), 'src'))
from utils.parameters import process_parameters_yaml
from models.k_bert import KBert
from models.bert import Bert

from utils.visualization import *

In [3]:
params = process_parameters_yaml()

In [4]:
class config:
    def __init__(self):
        self.dataset = 'agnews'
        self.model = 'kbert'
        self.batch_size = 2
        self.sample_data = 50
        self.overfit = False
#         self.checkpoint = 'model_best.ckpt'
        self.checkpoint = 'batch_size_4-max_epoch10-sample_None-overfit_False/epoch=6-step=167999-val_loss=0.24.ckpt'
cfg = config()

In [5]:
checkpoint_path = f'checkpoints/{cfg.model}/{cfg.checkpoint}'
model = KBert.load_from_checkpoint(checkpoint_path)
model.eval()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
lig = LayerIntegratedGradients(model, model.BertModel.embeddings)

In [8]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

def set_device(data):
    for i, d in enumerate(data):
        data[i] = d.to('cuda')
    return data

In [6]:
# sentences = ["Wales were eliminated in the second round of Euro 2020 as they were thrashed by Denmark, whose inspiring journey at the tournament continues to the quarter-finals. Backed by a fervent and almost exclusively Danish crowd, Kasper Hjulmand's side led as Kasper Dolberg curled in a fine 27th-minute strike from the edge of the penalty area. Dolberg struck again in the 48th minute as he seized on a defensive error from Wales substitute Neco Williams to smash in from close range."]
# sentences = ["Microsoft has officially announced Windows 11, its new operating system which will replace the current version over the next few years. Among all the new features are two seemingly small but related things that jumped out. First - Microsoft Teams, the video-calling app which saw a boom during 2020's pandemic, will be integrated into Windows 11 by default. And second - Skype will not be, for the first time in years. That seems to suggest that Teams is the new favourite child, and many pundits think this is the beginning of the end for what was once the king of calling apps."]
sentences = ["On Thursday, the Federal Reserve announced that all 23 banks subject to this year’s stress test easily passed. This was good news, widely anticipated, and sent the KBW Bank Index up 6.9% for the week, its best run since early February. The index is up 30% for the year. What comes next, however, should be even better news: The banks are now free from Fed pandemic restrictions to return capital to shareholders. Analysts at Barclay’s, for instance, expect the 20 banks in its coverage universe to return as much as $200 billion to shareholders in the next four quarters—double what they paid last year—giving investors plenty of reason to stick with the sector."]
sent_idx = 0
out = model.kg.add_knowledge(sentences)
token_id, mask, soft_idx, visible_matrix, depth, sentence_tree = out
model, token_id, mask, soft_idx, visible_matrix = set_device([model, token_id, mask, soft_idx, visible_matrix])

output = model.BertModel(input_ids=token_id, attention_mask=mask, position_ids=soft_idx)
logits = model.fc(output['pooler_output'])
pred = logits.argmax()
print(pred)

tensor(2, device='cuda:0')


In [10]:
ligs = []
attributions_sum = []
for i in range(4):
    attributions, delta =  lig.attribute(inputs=token_id, target=i, internal_batch_size=2,n_steps=100,additional_forward_args=( mask, soft_idx, visible_matrix),return_convergence_delta=True)
    ligs.append([attributions, delta])
    # print(torch.sum(attributions))
    att_sum = summarize_attributions(attributions)
    attributions_sum.append(att_sum)

In [11]:
indices = token_id[sent_idx].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

In [12]:
score_visualization = []
print('\033[1m', 'Visualization For Score', '\033[0m')

for i in range(4):
    att_sum = attributions_sum[i]
    score_vis = viz.VisualizationDataRecord(att_sum,
                                            int(pred),                        
                                            int(pred),
                                            i,
                                            sentences,
                                            att_sum.sum(),       
                                            all_tokens,
                                            delta)
    score_visualization.append(score_vis)
    print(viz.visualize_text([score_vis]))

 Visualization For Score 


<IPython.core.display.HTML object>


<IPython.core.display.HTML object>


<IPython.core.display.HTML object>


<IPython.core.display.HTML object>


In [ ]:
compute_attributions_sum(all_tokens, attributions_sum, pred, depth[sent_idx])

In [ ]:
# print(max(attributions_sum[pred]))
# print(min(attributions_sum[pred]))

tensor(0.6458, device='cuda:0', dtype=torch.float64)
tensor(-0.1262, device='cuda:0', dtype=torch.float64)


In [ ]:
print_color_sentence_tree(all_tokens, depth[0], soft_idx[0], attributions_sum[pred])

In [ ]:
dep_prev = 0
idx_prev = 0
token_list = ['[CLS]']
for token, dep, idx, att in zip(all_tokens, depth[0], soft_idx[0], attributions_sum[pred]):
    if token == '[PAD]':
        break
    token = att_color(token, att)
    # print(token)
    if dep == 0:
        print(token)
    elif dep == 1:
        print('\t'+token)
    elif dep == 2:
        print('\t\t'+token)

In [ ]:
print(max(attributions_sum[pred]))
print(min(attributions_sum[pred]))

In [25]:
tree = tree_structure(all_tokens, attributions_sum[pred], soft_idx, depth, sent_idx=0, threshold=(0.1, -0.03))
t = Tree.fromstring(tree)
t.pretty_print()